In [168]:
import sqlite3
import pandas as pd
import numpy as np

In [169]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [188]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [189]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [194]:
#Тест Nan values
data1 = data.copy()

data1.iloc[0:15] = np.NaN

data1.to_sql('boston1', con, index=False, if_exists='replace')

506

In [195]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [196]:
pd.read_sql(
    '''
    select coun - count(CRIM)  CRIM,
    coun - count(ZN)  ZN,
    coun - count(INDUS)  INDUS,
    coun - count(CHAS) CHAS,
    coun - count(NOX)  NOX
    from (
    SELECT CRIM, ZN, INDUS, CHAS, NOX ,
    count(*) over () as coun
    FROM boston1 b)
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,15,15,15,15,15


In [202]:
pd.read_sql(
    '''
    with cte as (SELECT count(*) over () as count_all
    FROM boston b)
    select 'CRIM' as columns, (select count_all from cte) - count(CRIM) as null_values
    from boston1
    union all
    select 'ZN',  (select count_all from cte) - count(ZN)
    from boston1
    union all
    select 'INDUS',(select count_all from cte) - count(INDUS)
    from boston1
    union all
    select 'CHAS', (select count_all from cte) - count(CHAS)
    from boston1
    union all
    select 'NOX',  (select count_all from cte) - count(NOX) 
    from boston1

    ''',
    con,
)

,columns,null_values
0,CRIM,15
1,ZN,15
2,INDUS,15
3,CHAS,15
4,NOX,15


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [203]:
pd.read_sql(
    '''

    SELECT count(distinct CRIM) CRIM, count(distinct ZN) ZN, count(distinct INDUS) INDUS, count(distinct CHAS) CHAS, count(distinct NOX) NOX
    FROM boston b
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


In [205]:
pd.read_sql(
    '''
    select 'CRIM' as columns, count(distinct CRIM) as nuniq_values
    from boston
    union all
    select 'ZN',   count(distinct ZN)
    from boston
    union all
    select 'INDUS',count(distinct INDUS)
    from boston
    union all
    select 'CHAS',  count(distinct CHAS)
    from boston
    union all
    select 'NOX',   count(distinct NOX)
    from boston
    ''',
    con,
)

,columns,nuniq_values
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [162]:

pd.read_sql(
    '''
    with cte as (select distinct count() over() count_all from boston)
    select * from (
    select distinct 'CRIM' column,
        case
            when (select count_all from cte)%2=0
                then (select avg(CRIM) over(order by crim rows between current row and 1 following) from boston limit 1 offset (select count_all/2-1 from cte))
                else (select avg(CRIM) over(order by crim rows current row) from boston limit 1 offset (select count_all/2 from cte))
                end as 'median',
    min(CRIM) min
    from boston
    union all
    select distinct 'ZN',
        case
            when (select count_all from cte)%2=0
                then (select avg(ZN) over(order by crim rows between current row and 1 following) from boston limit 1 offset (select count_all/2-1 from cte))
                else (select avg(ZN) over(order by crim rows current row) from boston limit 1 offset (select count_all/2 from cte))
                end as 'median',
    min(ZN)
    from boston
    union all
    select distinct 'INDUS',
        case
            when (select count_all from cte)%2=0
                then (select avg(INDUS) over(order by crim rows between current row and 1 following) from boston limit 1 offset (select count_all/2-1 from cte))
                else (select avg(INDUS) over(order by crim rows current row) from boston limit 1 offset (select count_all/2 from cte))
                end as 'median',
    min(INDUS)
    from boston
    union all
    select distinct 'CHAS',
        case
            when (select count_all from cte)%2=0
                then (select avg(CHAS) over(order by crim rows between current row and 1 following) from boston limit 1 offset (select count_all/2-1 from cte))
                else (select avg(CHAS) over(order by crim rows current row) from boston limit 1 offset (select count_all/2 from cte))
                end as 'median',
    min(CHAS)
    from boston
    union all
     select distinct 'NOX',
        case
            when (select count_all from cte)%2=0
                then (select avg(NOX) over(order by crim rows between current row and 1 following) from boston limit 1 offset (select count_all/2-1 from cte))
                else (select avg(NOX) over(order by crim rows current row) from boston limit 1 offset (select count_all/2 from cte))
                end as 'median',
    min(NOX)
    from boston
    )
    where median = min
    ''',
    con,
)

,column,median,min
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


В распределении данных колонок имеется правостороння ассиметрия. 50% значений имеют минимальное значение.

4. Выведитеdescribeнги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [202]:

pd.read_sql(
    '''
    select LSTAT,MEDV,rank_l  from (
    SELECT LSTAT,MEDV, max(medv) over() max_medv, rank() over(order by LSTAT) rank_l
    FROM boston)
    where medv = max_medv

 
    ''',
    con,
)

,LSTAT,MEDV,rank_l
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


В домах с самой дорогой стоимостью в основном проживает население с высоким социальным статусом. 

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [167]:

pd.read_sql(
    '''
    SELECT distinct 
    case 
    when CHAS = 0 
    then 'Не граничит'
    else 'Граничит' 
    end as CHAS, 
    round(avg(medv) over (partition by chas),2) avg_medv
    FROM boston
    ''',
    con,
)

,CHAS,avg_medv
0,Не граничит,22.09
1,Граничит,28.44


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [145]:

pd.read_sql(
    '''
    select * from (
    SELECT  distinct 'crim' as columns, (select avg(CRIM) over (partition by chas) from boston where chas = 1) - (select avg(CRIM) over (partition by chas) from boston where chas = 0) as diff
    from boston
    union all
    SELECT  distinct 'ZN', (select avg(ZN) over (partition by chas) from boston where chas = 1) - (select avg(ZN) over (partition by chas) from boston where chas = 0) as diff
    from boston
    union all
    SELECT  distinct 'INDUS', (select avg(INDUS) over (partition by chas) from boston where chas = 1) - (select avg(INDUS) over (partition by chas) from boston where chas = 0) as diff
    from boston
    union all
    SELECT  distinct 'NOX', (select avg(NOX) over (partition by chas) from boston where chas = 1) - (select avg(NOX) over (partition by chas) from boston where chas = 0) as diff
    from boston)

    where diff > 0

    ''',
    con,
)

,columns,diff
0,INDUS,1.699950
1,NOX,0.041609


В среднем концентрация оксидов азота незначительно выше у домов, граничащих с рекой. В среднем доля акров неторгового бизнеса также выше у домов, граничащих с рекой. 

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [249]:

pd.read_sql(
    '''
    select distinct perc,
    max(INDUS) over(partition by perc) indus,
    max(NOX) over(partition by perc) NOX
    from (
    select * from (
    SELECT INDUS, NOX, (ntile(10) over(order by indus))*10 perc
    FROM boston)
    )
    ''',
    con,
)

,perc,indus,NOX
0,10,2.89,0.538
1,20,4.39,0.647
2,30,5.96,0.515
3,40,7.38,0.507
4,50,9.69,0.585
5,60,13.89,0.585
6,70,18.10,0.770
7,80,18.10,0.740
8,90,19.58,0.871
9,100,27.74,0.871


С увеличением доли акров неторгового бизнеса на город прослеживаеся прямая корреляция с концентрацией оксидов азота.